In [1]:
import pandas as pd
import numpy as np
import string
import os
import re


/Users/sanjivjha/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
main_ds = pd.read_json('STAR_val.json')
main = main_ds.loc[:, ['question_id','question','video_id','start','end','answer', 'choices']]

In [3]:
df = pd.read_csv('/Users/sanjivjha/Developer/IMP-OIC/STAR_eval/all_tested.csv')
df['OIC_context']

0       The scene opens:\nFrom now on man_f89c wearing...
1       The scene opens:\nFrom now on glove_beed on ha...
2       The scene opens:\nFrom now on man_3171 wearing...
3       The scene opens:\nFrom now on woman_24bd weari...
4       The scene opens:\nFrom now on man_753c wearing...
                              ...                        
1569    The scene opens:\nFrom now on handle_a798 on d...
1570    The scene opens:\nFrom now on woman_1eeb has h...
1571    The scene opens:\nFrom now on man_3662 wearing...
1572    The scene opens:\nFrom now on woman_5121 weari...
1573    The scene opens:\nFrom now on man_cb8b has hai...
Name: OIC_context, Length: 1574, dtype: object

In [4]:
df['OIC_answer_without_CUID'] =  df['OIC_context'].apply(lambda x: re.sub('_[a-zA-Z0-9]*', '', x))
df['OIC_answer_without_CUID'][0]

'The scene opens:\nFrom now on man wearing shirt\nFrom now on man wearing hat\nFrom now on man wearing pant\nFrom now on man wearing glove\nFrom now on man has head\nFrom now on man has arm\nFrom now on man has ear\nFrom now on man wearing glass\nFrom now on shirt on man\nFrom now on hat on man\nFrom now on pant on man\nman has ear is not longer actual.\npant on man is not longer actual.\nAfter 1 seconds:\nAfter 2 seconds:\nFrom now on man has ear\nFrom now on hat on head\nFrom now on pant on man\nAfter 3 seconds:\nman wearing shirt is not longer actual.\nman has ear is not longer actual.\nshirt on man is not longer actual.\nhat on man is not longer actual.\nhat on head is not longer actual.\npant on man is not longer actual.\nAfter 4 seconds:\nFrom now on man has hair\nFrom now on man sitting on toilet\nFrom now on man wearing tie\nFrom now on man wearing jean\nFrom now on hair of man\nFrom now on ear of man\nFrom now on tie on man\nFrom now on hand of man\nFrom now on person wearing 

In [5]:
class Substitutable(str):
  def __new__(cls, *args, **kwargs):
    newobj = str.__new__(cls, *args, **kwargs)
    newobj.sub = lambda fro,to: Substitutable(re.sub(fro, to, newobj))
    return newobj

In [6]:
df['OIC_answer_without_CUID_temp'] = df['OIC_answer_without_CUID']
df['OIC_answer_without_CUID_temp'] = df['OIC_answer_without_CUID_temp'].apply(lambda x: Substitutable(x).sub('\n', ' ').sub('The scene opens:', ' ').sub('From now on', ' ').sub('is not longer actual.',' ').sub('\nAfter [0-9]* seconds:', ' '))

df['OIC_answer_without_CUID_temp']

0           man wearing shirt   man wearing hat   man ...
1           glove on hand   man wearing pant   man wea...
2           man wearing shirt   man has hair   man wea...
3           woman wearing shirt   woman has hair   wom...
4           man wearing shirt   man wearing pant   man...
                              ...                        
1569        handle on door   door has handle After 1 s...
1570        woman has hair   woman wearing shirt   wom...
1571        man wearing shirt   man wearing pant   man...
1572        woman wearing glass   woman in shirt   wom...
1573        man has hair   man wearing shirt   man sit...
Name: OIC_answer_without_CUID_temp, Length: 1574, dtype: object

In [7]:
df.head()

,Unnamed: 0.2,Unnamed: 0,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,OIC_question,Match,Unnamed: 0.1,OIC_answer_without_CUID,OIC_answer_without_CUID_temp
0,28920,8,Interaction_T1_76,Which object was thrown by the person?,XO8NL,3.6,12.5,The pillow.,"[{'choice_id': 0, 'choice': 'The shoe.', 'choi...",The scene opens:\nFrom now on man_f89c wearing...,3.0,Which object was thrown by the person? Guess t...,Correct,2.0,The scene opens:\nFrom now on man wearing shir...,man wearing shirt man wearing hat man ...
1,28921,9,Interaction_T1_78,Which object was opened by the person?,OY3LS,2.5,10.1,The laptop.,"[{'choice_id': 0, 'choice': 'The laptop.', 'ch...",The scene opens:\nFrom now on glove_beed on ha...,3.0,Which object was opened by the person? Guess t...,Wrong,3.0,The scene opens:\nFrom now on glove on hand\nF...,glove on hand man wearing pant man wea...
2,28922,13,Interaction_T1_101,Which object was put down by the person?,RG0KS,17.4,27.0,The box.,"[{'choice_id': 0, 'choice': 'The cup/glass/bot...",The scene opens:\nFrom now on man_3171 wearing...,4.0,Which object was put down by the person? Guess...,Wrong,4.0,The scene opens:\nFrom now on man wearing shir...,man wearing shirt man has hair man wea...
3,28923,17,Interaction_T1_137,Which object was closed by the person?,NUKJ0,19.4,26.6,The book.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",The scene opens:\nFrom now on woman_24bd weari...,3.0,Which object was closed by the person? Guess t...,Wrong,5.0,The scene opens:\nFrom now on woman wearing sh...,woman wearing shirt woman has hair wom...
4,28924,21,Interaction_T1_171,Which object was put down by the person?,LBRYS,8.9,13.4,The shoe.,"[{'choice_id': 0, 'choice': 'The sandwich.', '...",The scene opens:\nFrom now on man_753c wearing...,2.0,Which object was put down by the person? Guess...,Wrong,6.0,The scene opens:\nFrom now on man wearing shir...,man wearing shirt man wearing pant man...
